In [180]:
import pandas as pd
import pprint
from pymongo import MongoClient

In [126]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient.nfl_teams
collection = mydb.collection

In [127]:
nfl_df = pd.DataFrame(list(collection.find()))

In [128]:
nfl_df.head()

,Active Players,Base Salary,Cap %,Cap Hit,Dead Cap,Misc,Option Bonus,Pos,Pos Category,Pos Side,Restruc Bonus,Roster Bonus,Signing Bonus,Team,Workout Bonus,_id
0,Tyrod Taylor,"$10,000,000",6.78,"$16,000,000","($16,000,000)",-,-,QB,Passing,Offense,-,"$6,000,000",-,Cleveland Browns,-,5c3a1ff9492a821a0472582f
1,Baker Mayfield,"$480,000",2.52,"$5,942,360","($32,682,980)",-,-,QB,Passing,Offense,-,-,"$5,462,360",Cleveland Browns,-,5c3a1ff9492a821a04725830
2,Drew Stanton,"$1,150,000",0.96,"$2,265,625","($4,250,000)",-,-,QB,Passing,Offense,-,"$15,625","$1,000,000",Cleveland Browns,"$100,000",5c3a1ff9492a821a04725831
3,Andrew Luck,"$12,000,000",12.57,"$24,400,000","($37,200,000)",-,-,QB,Passing,Offense,-,"$6,000,000","$6,400,000",Indianapolis Colts,-,5c3a1ff9492a821a04725832
4,Jacoby Brissett,"$735,076",0.39,"$760,076","($25,000)",-,-,QB,Passing,Offense,-,-,-,Indianapolis Colts,"$25,000",5c3a1ff9492a821a04725833


In [129]:
nfl_df['Pos Category'].unique()

array(['Passing', 'Linebacker', 'Offensive Line', 'Defensive Line',
       'Secondary', 'Rushing', 'Special Teams'], dtype=object)

In [144]:
nfl_df = nfl_df.rename(columns={"Cap Hit": "Cap", "Pos Category": "Category"})

In [147]:
nfl_df.Cap = [x.strip('$') for x in nfl_df.Cap]

AttributeError: 'float' object has no attribute 'strip'

In [148]:
nfl_df["Cap"] = nfl_df["Cap"].str.replace(',', '')
nfl_df["Cap"] = nfl_df["Cap"].str.replace('-', '')

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [149]:
nfl_df["Cap"].head()

0    16000000.0
1     5942360.0
2     2265625.0
3    24400000.0
4      760076.0
Name: Cap, dtype: float64

In [150]:
nfl_df["Cap"] = pd.to_numeric(nfl_df["Cap"])

In [151]:
team_df = nfl_df.loc[:,["Cap", "Category", "Team"]]

In [152]:
team_df["Team"].value_counts()

Los Angeles Chargers    55
Houston Texans          54
Dallas Cowboys          54
Buffalo Bills           53
Denver Broncos          53
Philadelphia Eagles     53
Miami Dolphins          53
Indianapolis Colts      53
Los Angeles Rams        53
Arizona Cardinals       53
Detroit Lions           53
Jacksonville Jaguars    53
Pittsburgh Steelers     53
Carolina Panthers       53
Oakland Raiders         53
New York Jets           53
New York Giants         53
Chicago Bears           53
Cleveland Browns        53
New England Patriots    53
Minnesota Vikings       53
Atlanta Falcons         53
Green Bay Packers       53
Kansas City Chiefs      53
Tampa Bay Buccaneers    53
Cincinnati Bengals      53
Seattle Seahawks        53
San Francisco 49ers     53
Baltimore Ravens        53
Washington Redskins     53
Tennessee Titans        53
New Orleans Saints      52
Name: Team, dtype: int64

In [161]:
team_group = team_df.groupby(["Team", "Category"])

In [162]:
team_group['Cap'].dtype

Team                  Category      
Arizona Cardinals     Defensive Line    float64
                      Linebacker        float64
                      Offensive Line    float64
                      Passing           float64
                      Rushing           float64
                      Secondary         float64
                      Special Teams     float64
Atlanta Falcons       Defensive Line    float64
                      Linebacker        float64
                      Offensive Line    float64
                      Passing           float64
                      Rushing           float64
                      Secondary         float64
                      Special Teams     float64
Baltimore Ravens      Defensive Line    float64
                      Linebacker        float64
                      Offensive Line    float64
                      Passing           float64
                      Rushing           float64
                      Secondary         float64
   

In [163]:
team_group.describe()

Cap                              \
                                    count          mean           std   
Team                 Category                                           
Arizona Cardinals    Defensive Line   9.0  2.505404e+06  4.940793e+06   
                     Linebacker       6.0  2.252244e+06  3.341766e+06   
                     Offensive Line   8.0  3.253411e+05  1.921913e+05   
                     Passing         13.0  2.595097e+06  4.724353e+06   
                     Rushing          4.0  1.698979e+06  2.229288e+06   
                     Secondary       10.0  2.604226e+06  4.563491e+06   
                     Special Teams    3.0  1.224828e+06  1.267044e+06   
Atlanta Falcons      Defensive Line  10.0  2.057922e+06  1.501367e+06   
                     Linebacker       6.0  7.527813e+05  2.720874e+05   
                     Offensive Line   8.0  3.898972e+06  4.168644e+06   
                     Passing         11.0  4.422097e+06  5.894018e+06   
                     Rushing          4.0  5.136022e+05  3.696668e+05   
                     Secondary       10.0  2.897888e+06  4.487482e+06   
                     Special Teams    4.0  1.343505e+06  1.280021e+06   
Baltimore Ravens     Defensive Line   6.0  1.593555e+06  2.130973e+06   
                     Linebacker      10.0  2.374938e+06  2.946649e+06   
                     Offensive Line   8.0  2.600291e+06  3.513794e+06   
                     Passing         12.0  3.663625e+06  6.764958e+06   
                     Rushing          4.0  5.647890e+05  2.420314e+05   
                     Secondary       10.0  4.008722e+06  4.382665e+06   
                     Special Teams    3.0  3.245555e+06  1.971047e+06   
Buffalo Bills        Defensive Line   9.0  3.601545e+06  3.529169e+06   
                     Linebacker       5.0  1.620229e+06  1.382446e+06   
                     Offensive Line   9.0  1.264468e+06  8.342932e+05   
                     Passing         13.0  1.394949e+06  2.494773e+06   
                     Rushing          4.0  3.372739e+06  3.855764e+06   
                     Secondary       10.0  1.674552e+06  2.168646e+06   
                     Special Teams    3.0  1.246765e+06  1.529558e+06   
Carolina Panthers    Defensive Line  10.0  4.351805e+06  5.333551e+06   
                     Linebacker       6.0  2.413526e+06  3.009426e+06   
...                                   ...           ...           ...   
San Francisco 49ers  Secondary       10.0  1.203536e+06  2.067902e+06   
                     Special Teams    3.0  1.044713e+06  1.089293e+06   
Seattle Seahawks     Defensive Line  10.0  1.062402e+06  7.025244e+05   
                     Linebacker       6.0  4.245685e+06  5.497235e+06   
                     Offensive Line   9.0  2.486358e+06  2.811166e+06   
                     Passing          8.0  5.718905e+06  8.205206e+06   
                     Rushing          6.0  8.637673e+05  6.733381e+05   
                     Secondary       10.0  1.224576e+06  1.120531e+06   
                     Special Teams    3.0  1.065846e+06  8.229038e+05   
Tampa Bay Buccaneers Defensive Line  10.0  4.896899e+06  4.706776e+06   
                     Linebacker       5.0  2.236118e+06  3.652929e+06   
                     Offensive Line   8.0  2.862275e+06  3.954641e+06   
                     Passing         13.0  4.154516e+06  5.512418e+06   
                     Rushing          3.0  1.171934e+06  4.949236e+05   
                     Secondary       11.0  1.210521e+06  2.265050e+06   
                     Special Teams    3.0  1.296471e+06  1.486890e+06   
Tennessee Titans     Defensive Line   5.0  2.375881e+06  2.185610e+06   
                     Linebacker      12.0  2.548419e+06  3.097740e+06   
                     Offensive Line   9.0  2.974029e+06  3.589071e+06   
                     Passing         11.0  1.828931e+06  2.517426e+06   
                     Rushing          3.0  1.972659e+06  2.134930e+06   
                     Se

In [200]:
team_list = list(team_df['Team'].unique())
team_list.sort()
print(team_list)

['Arizona Cardinals', 'Atlanta Falcons', 'Baltimore Ravens', 'Buffalo Bills', 'Carolina Panthers', 'Chicago Bears', 'Cincinnati Bengals', 'Cleveland Browns', 'Dallas Cowboys', 'Denver Broncos', 'Detroit Lions', 'Green Bay Packers', 'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars', 'Kansas City Chiefs', 'Los Angeles Chargers', 'Los Angeles Rams', 'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots', 'New Orleans Saints', 'New York Giants', 'New York Jets', 'Oakland Raiders', 'Philadelphia Eagles', 'Pittsburgh Steelers', 'San Francisco 49ers', 'Seattle Seahawks', 'Tampa Bay Buccaneers', 'Tennessee Titans', 'Washington Redskins']


In [166]:
team_cap = team_group["Cap"].sum()

In [167]:
teamp_cap.to_csv('team_cap.csv')